In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os

This notebook walks through processing the CDS downloads to get in a standard format by lat x lon cell, for the entire time period of 2000-2021. 

In [2]:
years = np.arange(2000,2022)
fp = '/mnt/d/ERA5_hourly/'
var = '2m_temperature'
region = '01'
rgi_fp = '/home/claire/research/RGI/rgi60/00_rgi60_attribs/'

First, open the example data and retrieve all glacier lat/lon pairs for the region. Get the nearest neighbors from the GCM data and filter out to only get unique pairs.

In [3]:
data = xr.open_dataset(fp+'GCMexample.nc')
datalat = data.coords['latitude'][:].values
datalon = data.coords['longitude'][:].values - 180

for fn in os.listdir(rgi_fp):
    if region in fn:
        rgi_fn = os.path.join(rgi_fp,fn)
main_glac_rgi = pd.read_csv(rgi_fn, encoding = 'ISO-8859-1',index_col='RGIId')

region_coords = []
lat_nearidx = (np.abs(main_glac_rgi['CenLat'].values[:,np.newaxis] - datalat).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi['CenLon'].values[:,np.newaxis] - datalon).argmin(axis=1))

latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = list(set(latlon_nearidx))
for latlonidx in latlon_nearidx_unique:
    lat = data.coords['latitude'][latlonidx[0]].values - 0
    lon = data.coords['longitude'][latlonidx[1]].values - 180

    if lon < -133.999:
        region_coords.append([lat,lon])

In [4]:
lat_min = 50
lat_max = 72
lon_min = -180
lon_max = -134
region_coords_rounded = np.floor(np.array(region_coords))

for lat in np.arange(lat_min,lat_max+1):
    where_lat = np.where(np.abs(region_coords_rounded[:,0] - lat) < 1e-3)
    for lon in np.arange(lon_min,lon_max+1):
        where_lon = np.where(np.abs(region_coords_rounded[:,1] - lon) < 1e-3)
        intersect = np.intersect1d(where_lat,where_lon)

        if len(intersect) > 0:
            cell_coords = np.array(region_coords)[intersect].round(1)
            print(cell_coords)
            for year in years:
                ds = xr.open_dataset(f'{fp}{var}/ERA5_{var}_AK{year}_hourly.nc')
                if year == years[0]:
                    ds_latlon = ds.sel(latitude=cell_coords[:,0],longitude=cell_coords[:,1])
                else:
                    ds_latlon = xr.concat([ds_latlon,ds.sel(latitude=cell_coords[:,0],longitude=cell_coords[:,1])],dim='time')
                print(year,ds_latlon)
            ds_latlon.to_netcdf(f'{fp}gridded/{var}/ERA5_hourly_{var}_{lat}_{lon}.nc')

[[  52.1 -176.1]]
2000 <xarray.Dataset>
Dimensions:    (longitude: 1, latitude: 1, time: 8784)
Coordinates:
  * longitude  (longitude) float32 -176.1
  * latitude   (latitude) float32 52.1
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-07-25 18:27:29 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...


2001 <xarray.Dataset>
Dimensions:    (longitude: 1, latitude: 1, time: 17544)
Coordinates:
  * longitude  (longitude) float32 -176.1
  * latitude   (latitude) float32 52.1
  * time       (time) datetime64[ns] 2000-01-01 ... 2001-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
Attributes:
    Conventions:  CF-1.6
    history:      2023-07-25 18:27:29 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...
2002 <xarray.Dataset>
Dimensions:    (longitude: 1, latitude: 1, time: 26304)
Coordinates:
  * longitude  (longitude) float32 -176.1
  * latitude   (latitude) float32 52.1
  * time       (time) datetime64[ns] 2000-01-01 ... 2002-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
Attributes:
    Conventions:  CF-1.6
    history:      2023-07-25 18:27:29 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...
2003 <xarray.Dataset>
Dimensions:    (longitude: 1, latitude: 1, time: 35064)
Coordina

KeyboardInterrupt: 

In [17]:
lat = 52
lon = -177
year = 2005
var = '2m_dewpoint_temperature'
# fpopen = f'{fp}gridded/{var}/ERA5_hourly_{var}_{lat}_{lon}.nc'
fpopen = f'{fp}{var}/ERA5_{var}_AK{year}_hourly.nc'
ds = xr.open_dataset(fpopen)
print(ds.sel(latitude=72.0,longitude=-180.0,time='2005-12-31T23:00:00').d2m.values)
    

nan


In [22]:
ds = xr.open_dataset('/mnt/d/ERA5_hourly/2m_temperature/ERA5_2m_temperature_AK2004_hourly.nc')
print(ds.sel(latitude=54.0,longitude=-134,time='2004-08-31T23:00:00').t2m.values)

nan
